# Using pretrained GloVe Embedding

In [1]:
# Based on
# https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/6.1-using-word-embeddings.ipynb
# https://machinelearningmastery.com/develop-word-embeddings-python-gensim/

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.8.0


### Download data:
* original imdb database from http://ai.stanford.edu/~amaas/data/sentiment/
* pre-computed GloVe embeddings from on Wikipedia data (6B) https://nlp.stanford.edu/projects/glove/

In [5]:
import os

imdb_dir = 'C:/Users/olive/Development/data/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname), encoding='UTF-8')
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [6]:
len(texts)

25000

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 500  # We will cut reviews after 100 words
training_samples = 15000  # We will be training on 200 samples
validation_samples = 10000  # We will be validating on 10000 samples
max_words = 10000  # We will only consider the top 10,000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

Using TensorFlow backend.


Found 88582 unique tokens.
Shape of data tensor: (25000, 500)
Shape of label tensor: (25000,)


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42, stratify=labels)

In [10]:
x_train.shape

(20000, 500)

## With pre-defined and fixed embeddings, we can not be better than just guessing 

In [11]:
glove_dir = 'C:/Users/olive/Development/data/glove.6B'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'), encoding='UTF-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [12]:
embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [13]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [14]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [15]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          1000000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 50000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1600032   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 2,600,065
Trainable params: 1,600,065
Non-trainable params: 1,000,000
_________________________________________________________________


In [18]:
batch_size=1000
model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=batch_size,
                    validation_split=0.2)

Train on 16000 samples, validate on 4000 samples
Epoch 1/10
16000/16000 [==============================] - 1s 48us/step - loss: 0.9644 - acc: 0.5029 - val_loss: 0.6932 - val_acc: 0.4945
Epoch 2/10
16000/16000 [==============================] - 0s 24us/step - loss: 0.6931 - acc: 0.5014 - val_loss: 0.6932 - val_acc: 0.4945
Epoch 3/10
16000/16000 [==============================] - 0s 23us/step - loss: 0.6931 - acc: 0.5014 - val_loss: 0.6932 - val_acc: 0.4945
Epoch 4/10
16000/16000 [==============================] - 0s 23us/step - loss: 0.6931 - acc: 0.5014 - val_loss: 0.6932 - val_acc: 0.4945
Epoch 5/10
16000/16000 [==============================] - 0s 23us/step - loss: 0.6931 - acc: 0.5014 - val_loss: 0.6932 - val_acc: 0.4945
Epoch 6/10
16000/16000 [==============================] - 0s 22us/step - loss: 0.6931 - acc: 0.5014 - val_loss: 0.6932 - val_acc: 0.4945
Epoch 7/10
16000/16000 [==============================] - 0s 23us/step - loss: 0.6931 - acc: 0.5014 - val_loss: 0.6932 - val_acc:

## Embedding trainable, but still pre-set

In [16]:
model.layers[0].trainable = True

In [17]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          1000000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 50000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1600032   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 2,600,065
Trainable params: 2,600,065
Non-trainable params: 0
_________________________________________________________________


In [19]:
batch_size=1000
model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=batch_size,
                    validation_split=0.2)

Train on 16000 samples, validate on 4000 samples
Epoch 1/100
16000/16000 [==============================] - 1s 42us/step - loss: 0.6872 - acc: 0.5368 - val_loss: 0.6789 - val_acc: 0.5817
Epoch 2/100
16000/16000 [==============================] - 1s 38us/step - loss: 0.6610 - acc: 0.6124 - val_loss: 0.6483 - val_acc: 0.6625
Epoch 3/100
16000/16000 [==============================] - 1s 38us/step - loss: 0.6072 - acc: 0.7096 - val_loss: 0.6066 - val_acc: 0.7432
Epoch 4/100
16000/16000 [==============================] - 1s 39us/step - loss: 0.5435 - acc: 0.7864 - val_loss: 0.5704 - val_acc: 0.7510
Epoch 5/100
16000/16000 [==============================] - 1s 39us/step - loss: 0.4832 - acc: 0.8464 - val_loss: 0.5604 - val_acc: 0.7502
Epoch 6/100
16000/16000 [==============================] - 1s 38us/step - loss: 0.4364 - acc: 0.8869 - val_loss: 0.5564 - val_acc: 0.7625
Epoch 7/100
16000/16000 [==============================] - 1s 39us/step - loss: 0.4021 - acc: 0.9178 - val_loss: 0.5374 - v

16000/16000 [==============================] - 1s 51us/step - loss: 0.1589 - acc: 0.9921 - val_loss: 0.6605 - val_acc: 0.7913
Epoch 60/100
16000/16000 [==============================] - 1s 44us/step - loss: 0.1573 - acc: 0.9921 - val_loss: 0.6617 - val_acc: 0.7925
Epoch 61/100
16000/16000 [==============================] - 1s 38us/step - loss: 0.1556 - acc: 0.9921 - val_loss: 0.6859 - val_acc: 0.7860
Epoch 62/100
16000/16000 [==============================] - 1s 39us/step - loss: 0.1541 - acc: 0.9921 - val_loss: 0.6506 - val_acc: 0.7943
Epoch 63/100
16000/16000 [==============================] - 1s 39us/step - loss: 0.1525 - acc: 0.9921 - val_loss: 0.6791 - val_acc: 0.7890
Epoch 64/100
16000/16000 [==============================] - 1s 38us/step - loss: 0.1510 - acc: 0.9921 - val_loss: 0.6697 - val_acc: 0.7910
Epoch 65/100
16000/16000 [==============================] - 1s 39us/step - loss: 0.1496 - acc: 0.9921 - val_loss: 0.6740 - val_acc: 0.7903
Epoch 66/100
16000/16000 [==============

## Embeddings trained from scratch

In [20]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 100)          1000000   
_________________________________________________________________
flatten_2 (Flatten)          (None, 50000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1600032   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 2,600,065
Trainable params: 2,600,065
Non-trainable params: 0
_________________________________________________________________


In [21]:
batch_size=1000
model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=batch_size,
                    validation_split=0.2)

Train on 16000 samples, validate on 4000 samples
Epoch 1/10
16000/16000 [==============================] - 1s 60us/step - loss: 0.7110 - acc: 0.5105 - val_loss: 0.6878 - val_acc: 0.5138
Epoch 2/10
16000/16000 [==============================] - 1s 38us/step - loss: 0.6634 - acc: 0.5683 - val_loss: 0.6735 - val_acc: 0.5337
Epoch 3/10
16000/16000 [==============================] - 1s 39us/step - loss: 0.6034 - acc: 0.6673 - val_loss: 0.5625 - val_acc: 0.7602
Epoch 4/10
16000/16000 [==============================] - 1s 41us/step - loss: 0.4082 - acc: 0.8406 - val_loss: 0.3716 - val_acc: 0.8425
Epoch 5/10
16000/16000 [==============================] - 1s 46us/step - loss: 0.2298 - acc: 0.9159 - val_loss: 0.2989 - val_acc: 0.8720
Epoch 6/10
16000/16000 [==============================] - 1s 38us/step - loss: 0.1364 - acc: 0.9556 - val_loss: 0.2882 - val_acc: 0.8760
Epoch 7/10
16000/16000 [==============================] - 1s 37us/step - loss: 0.0784 - acc: 0.9809 - val_loss: 0.2911 - val_acc:

In [23]:
train_loss, train_accuracy = model.evaluate(x_train, y_train, batch_size=batch_size)
train_accuracy

20000/20000 [==============================] - 0s 17us/step


0.9756000012159347

In [24]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, batch_size=batch_size)
test_accuracy

5000/5000 [==============================] - 0s 16us/step


0.8834000110626221